# 日本の年齢別収入データ分析ガイド

このJupyterノートブックは、日本の公的統計データから年齢別収入データを取得・処理するためのサンプルコードです。複数の統計調査から年齢別の収入情報を取得し、将来の収入予測に使用できる形式に変換します。

## 概要

### 使用するデータソース

| 調査名 | 実施機関 | データの種類 | 年齢別情報 | 備考 |
|--------|----------|--------------|------------|------|
| 賃金構造基本統計調査 | 厚生労働省 | 賃金（所定内・外、賞与など） | ◎ あり | 最も詳細な個人ベースの賃金統計。年齢×学歴×勤続年数×職種など多変量分析可 |
| 家計調査 | 総務省統計局 | 世帯の実収入・支出 | ○ あり | 勤労者世帯の世帯主年齢別の平均月収がわかる。世帯単位のデータ |
| 全国家計構造調査（旧・全国消費実態調査） | 総務省統計局 | 所得・支出・貯蓄・負債等 | ○ あり | 5年ごと実施。世帯主年齢別に詳細な所得内訳がわかる |
| 民間給与実態統計調査 | 国税庁 | 年間給与（所得） | ◎ あり | 全国の給与所得者の年収実態。平均・中央値・分布あり。正規/非正規別も可 |
| 国民生活基礎調査 | 厚生労働省 | 世帯の所得・生活実態 | ○ あり | 世帯主年齢別の所得平均・中央値・分布あり。福祉・医療と併せた分析が可能 |
| 毎月勤労統計調査 | 厚生労働省 | 月次の給与・労働時間 | ○ 一部あり | 年報などで年齢別給与も公表。短期の変化や景気動向把握に向く |
| 労働力調査 | 総務省統計局 | 雇用状況（就業・失業） | ○（人数のみ） | 年齢別の就業者数・非労働力人口は把握できるが、賃金金額はなし |
| 就業構造基本調査 | 総務省統計局 | 職業・就業形態・月収階級等 | △（金額なし、階級） | 月収階級別の年齢分布あり。職種・雇用形態とのクロス集計が可能 |

## 環境セットアップ

### 必要なライブラリのインポート

In [ ]:
import os
import sys
from pathlib import Path
import requests
import pandas as pd
from dotenv import load_dotenv

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

### パッケージのパス設定とインポート

In [ ]:
# パッケージのパスを追加（exampleフォルダから一つ上の階層のfpy_datareaderを参照）
sys.path.append(os.path.dirname(os.getcwd()))

from jpy_datareader.estat import MetaInfoReader, StatsDataReader
from fpy_datareader.future_income_estimator import (
    add_age_midpoint_column, 
    interpolate_age_income,
    add_income_multiplier_columns,
    sum_income_multiplier,
    calculate_lifetime_income,
    create_age_income_dataframe_with_midpoint,
    process_family_income_expenditure_survey_data,
    process_family_income_consumption_wealth_survey_data,
    process_wage_structure_survey_data,
    process_wage_structure_survey_data_by_prefecture,
    process_living_conditions_survey_data,
    create_age_income_table
)

### APIキーの設定

In [ ]:
# APIキーの読み込み
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

**重要**: e-Stat APIを使用するには、事前にAPIキーの取得が必要です。[e-Stat API](https://www.e-stat.go.jp/api/)で登録してください。

## 1. 家計調査

家計調査は、総務省統計局が実施する世帯の収入・支出を調べる基幹統計です。

### データの特徴
- **対象**: 二人以上の世帯および単身世帯
- **頻度**: 月次調査
- **収入データ**: 世帯主の年齢階級別平均月収
- **用途**: 世帯単位での収入分析

### 利用可能なstatsDataId

#### 二人以上の世帯
- **月次データ**: `0002070010` - 用途分類（世帯主の年齢階級別）
- **年次データ**: `0002070011` - 用途分類（世帯主の年齢階級別）年次

#### 単身世帯（支出のみ）
- **月次データ**: `0003000795` - 用途分類（年齢階級別）
- **年次データ**: `0002190004` - 用途分類（年齢階級別）年次

**注意**: 単身世帯のデータ（statsDataId: `0003000795`, `0002190004`）は支出項目のみで、収入データは含まれていません。収入分析には二人以上世帯のデータ（statsDataId: `0002070010`, `0002070011`）を使用してください。

### メタデータの確認

In [ ]:
# 二人以上世帯の年次データを例として使用
kakei_setai_statsDataId = "0002070011"
kakei_setai_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=kakei_setai_statsDataId,
)
kakei_setai_meta = kakei_setai_metainfo.read_class_objs()
kakei_setai_metainfo.OVERALL_TOTAL_NUMBER

#### メタデータの構造確認

各メタデータの内容を順次確認します：

1. **表章項目** (`kakei_setai_meta[0]`): 金額、世帯数、人員などの測定項目
2. **用途分類** (`kakei_setai_meta[1]`): 収入・支出の詳細分類
3. **世帯区分** (`kakei_setai_meta[2]`): 二人以上世帯、勤労者世帯など
4. **年齢階級** (`kakei_setai_meta[3]`): 世帯主の年齢区分（25～29歳、30～34歳など）
5. **地域区分** (`kakei_setai_meta[4]`): 全国、地域別
6. **時間軸** (`kakei_setai_meta[5]`): 調査年月


In [ ]:
kakei_setai_meta[0]

In [ ]:
kakei_setai_meta[1]['meta_dataframe']

In [ ]:
kakei_setai_meta[2]

In [ ]:
kakei_setai_meta[3]

In [ ]:
kakei_setai_meta[4]

In [ ]:
kakei_setai_meta[5]

### データ処理と結果

この関数は、家計調査データから年齢別収入カーブを作成し、以下の列を持つDataFrameを返します：
- `age`: 年齢（15歳から開始）
- `income`: 年収（円）
- `growth_rate`: 前年からの成長率
- `growth_rate_ma`: 移動平均された成長率

In [ ]:
process_family_income_expenditure_survey_data(
    api_key=appId,
).head(30)


## 2. 全国家計構造調査

5年ごとに実施される詳細な家計調査で、より詳細な所得・資産情報を提供します。

### データの特徴
- **実施頻度**: 5年ごと
- **詳細度**: 所得・支出・貯蓄・負債の詳細分析が可能
- **最新データ**: 2019年調査結果

### 利用可能なstatsDataId

#### 主要なデータセット
- **基本データ**: `0003424621` - 世帯の種類・年齢階級・所得構成別データ
- 他にも以下のような詳細分類データが利用可能：
  - `0003426498` - 世帯の種類・世帯区分・性別・年齢階級・所得構成別
  - `0003426474` - 世帯の種類・企業規模・年齢階級・所得構成別
  - `0003426475` - 世帯人員・有業人員・年齢階級・所得構成別
  - `0003426476` - 世帯の種類・職業・年齢階級・所得構成別
  - `0003426478` - 世帯の種類・公的年金受給額階級・年齢階級・所得構成別
  - `0003426483` - 世帯の種類・年間収入階級・年齢階級・所得構成別
  - `0003426484` - 世帯の種類・年間収入十分位階級・年齢階級・所得構成別
  - `0003426485` - 世帯の種類・年間収入五分位階級・年齢階級・所得構成別

### メタデータの確認


In [ ]:
ficw_statsDataId = "0003424621"
ficw_metainfo = MetaInfoReader(
    api_key=appId,
    statsDataId=ficw_statsDataId,
)
ficw_meta = ficw_metainfo.read_class_objs()
ficw_metainfo.OVERALL_TOTAL_NUMBER


#### 重要なメタデータ項目

1. **世帯の種類**: 総世帯、二人以上の世帯、単身世帯
2. **世帯区分**: 全世帯、勤労者世帯、無職世帯、その他の世帯
3. **性別**: 平均、男、女
4. **収支項目分類**: 総数、平均、支払、実支出、消費支出など406項目
5. **年齢階級**: 30歳未満～85歳以上の32区分


In [ ]:
ficw_meta[0]

In [ ]:
ficw_meta[1]

In [ ]:
ficw_meta[2]

In [ ]:
ficw_meta[3]

### 可処分所得の確認

In [ ]:
ficw_meta_df = ficw_meta[4]['meta_dataframe']
ficw_meta_df

In [ ]:
# 可処分所得のコードを確認
ficw_meta_df = ficw_meta[4]['meta_dataframe']
disposable_income = ficw_meta_df[ficw_meta_df["収支項目分類（細分類）406区分コード"].str[0]=="4"]

In [ ]:
ficw_meta[5]

In [ ]:
ficw_meta[6]

In [ ]:
ficw_meta[7]


### データ処理


In [ ]:
process_family_income_consumption_wealth_survey_data(
    api_key=appId,
).head(30)


## 3. 賃金構造基本統計調査
https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00450091&tstat=000001011429&cycle=0&tclass1val=0&metadata=1&data=1
厚生労働省が実施する最も詳細な個人レベルの賃金統計です。

### データの特徴
- **対象**: 常用労働者（10人以上の事業所）
- **詳細度**: 年齢×学歴×勤続年数×職種の多変量分析が可能
- **データ項目**: 年齢、勤続年数、所定内・超過実労働時間数、現金給与額など

### 利用可能なstatsDataId

#### 全国データ
- **産業・年齢階級別**: `0003425893` - 一般_産業大・中分類_年齢階級別DB

#### 都道府県別データ
- **都道府県・年齢階級別**: `0003426933` - 一般_都道府県別_年齢階級別DB

### 全国データの分析


In [ ]:
chingin_statsDataId = "0003425893"
chingin_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=chingin_statsDataId,
)
chingin_meta = chingin_metainfo.read_class_objs()
chingin_metainfo.OVERALL_TOTAL_NUMBER


#### メタデータの構造

1. **企業規模**: 10人以上、1,000人以上、100～999人、10～99人
2. **産業分類**: 109の詳細産業分類
3. **性別**: 男女計、男、女
4. **年齢階級**: ～19歳、20～24歳、...、70歳～の13区分
5. **学歴**: 中学、高校、専門学校、高専・短大、大学、大学院、不明
6. **民・公区分**: 民営事業所、民営+公営


In [ ]:
chingin_meta[0]

In [ ]:
chingin_meta[1]

In [ ]:
chingin_meta[2]["meta_dataframe"]

In [ ]:
chingin_meta[3]

In [ ]:
chingin_meta[4]

In [ ]:
chingin_meta[5]

In [ ]:
chingin_meta[6]

In [ ]:
chingin_meta[7]

### データ処理

In [ ]:
chingin_growth_df = process_wage_structure_survey_data(
    api_key=appId,
    start_age = 15,
    end_age = 100,
    terminal_start = 0,
    terminal_end = 115,
)
chingin_growth_df

In [ ]:
add_income_multiplier_columns(chingin_growth_df, rate_column='growth_rate_ma').to_csv("future_income_estimated_based_on_wage_structure_survey.csv", index=False)

### 都道府県別データ

In [ ]:
chingin_prefecture_statsDataId = "0003426933"
chingin_prefecture_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=chingin_prefecture_statsDataId,
)
chingin_prefecture_meta = chingin_prefecture_metainfo.read_class_objs()
chingin_prefecture_metainfo.OVERALL_TOTAL_NUMBER

都道府県別データには47都道府県の詳細な賃金情報が含まれています。

In [ ]:
chingin_prefecture_meta[0]

In [ ]:
chingin_prefecture_meta[1]

In [ ]:
chingin_prefecture_meta[2]

In [ ]:
chingin_prefecture_meta[3]

In [ ]:
chingin_prefecture_meta[4]

In [ ]:
chingin_prefecture_meta[5]

In [ ]:
chingin_prefecture_meta[6]

### データ処理

In [ ]:
process_wage_structure_survey_data_by_prefecture(
    api_key=appId
).head(30)


## 4. 国民生活基礎調査

厚生労働省が実施する世帯の所得・生活実態調査です。

### データの特徴
- **実施頻度**: 3年ごと（大規模調査）
- **特徴**: 福祉・医療データと組み合わせた分析が可能
- **所得データ**: 世帯主年齢別の平均・中央値・分布

### 利用可能なstatsDataId

#### 主要なデータセット
- **基本データ**: `0004021242` - 1世帯当たり平均所得金額・世帯人員1人当たり平均所得金額、世帯主の年齢（10歳階級）・年次別
- **詳細データ**: `0004021274` - 平均所得金額・平均世帯人員・平均有業人員、世帯主の年齢（10歳階級）別

### メタデータの確認


In [ ]:
seikatsu_statsDataId = "0004021242"
seikatsu_metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=seikatsu_statsDataId,
)
seikatsu_meta = seikatsu_metainfo.read_class_objs()
seikatsu_metainfo.OVERALL_TOTAL_NUMBER

#### 主要項目

1. **表章項目**: 1世帯当たり平均所得金額、世帯人員1人当たり平均所得金額
2. **年齢階級**: 29歳以下、30～39歳、40～49歳、50～59歳、60～69歳、70歳以上
3. **時系列データ**: 1985年から2022年までの長期データ

In [ ]:
seikatsu_meta[0]

In [ ]:
seikatsu_meta[1]

In [ ]:
seikatsu_meta[2]

In [ ]:
seikatsu_meta[3]

### データ処理

In [ ]:
process_living_conditions_survey_data(
    api_key=appId
).head(30)


## 5. 民間給与実態統計調査

国税庁が実施する給与所得者の年収実態調査です。

### データの特徴
- **実施機関**: 国税庁
- **対象**: 全国の給与所得者
- **データ内容**: 年間給与（所得）の詳細分析
- **特徴**: 平均・中央値・分布データ、正規/非正規雇用別分析が可能

### 利用可能なstatsDataId

#### 主要なデータセット
- **全国計表 第10表**: `0004009621` - 事業所規模別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者）（2014年～）
- **全国計表 第11表**: `0004009622` - 企業規模別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者）（2014年～）
- **全国計表 第12表**: `0004009623` - 業種別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者）（2014年～）
- **全国計表 第12表**: `0004009625` - 業種別及び年齢階層別の給与所得者数・給与額（1年を通じて勤務した給与所得者（乙欄適用者を除く））（2014年～）




## データの統合と活用

### 処理済みデータの特徴

各処理関数は、以下の共通フォーマットでデータを返します：

- **age**: 年齢（15歳から開始）
- **income**: 年収（円）
- **growth_rate**: 年収の成長率
- **growth_rate_ma**: 移動平均された成長率


## 注意事項

### メタデータの確認の重要性

**すべてのメタデータ確認処理は削除しないでください。** これらの処理は以下の理由で重要です：

1. **データ構造の理解**: 各調査の項目分類や階層構造を把握
2. **適切なパラメータ設定**: API呼び出し時の正しいコード指定
3. **データ品質の確保**: 期待するデータが取得できているかの確認
4. **再現性の保証**: 分析結果の再現と検証のための記録

### API使用時の注意

- e-Stat APIには利用制限があります
- 大量データ取得時は適切な間隔を空けてください
- APIキーは適切に管理してください

### データの解釈

- 各調査は対象範囲や調査手法が異なります
- 複数の調査結果を比較する際は、それぞれの特性を理解してください
- 時系列データを扱う際は、調査手法の変更や経済情勢の影響を考慮してください

## まとめ

このコードは、日本の公的統計データから年齢別収入データを体系的に取得・処理するためのツールです。複数の統計調査を組み合わせることで、より包括的で信頼性の高い年齢別収入分析が可能になります。